In [25]:
import sqlite3
import matplotlib
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import utility as util
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, f1_score


In [26]:
neo_bloog_gas_all_time = pd.read_csv('neo_bloog_gas_all_time.csv')

In [27]:
neo_bloog_gas_all_time.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'charttime', 'SPECIMEN', 'AADO2',
       'BASEEXCESS', 'BICARBONATE', 'TOTALCO2', 'CARBOXYHEMOGLOBIN',
       'CHLORIDE', 'CALCIUM', 'GLUCOSE', 'HEMATOCRIT', 'HEMOGLOBIN',
       'INTUBATED', 'LACTATE', 'METHEMOGLOBIN', 'O2FLOW', 'FIO2', 'SO2',
       'PCO2', 'PEEP', 'PH', 'PO2', 'POTASSIUM', 'REQUIREDO2', 'SODIUM',
       'TEMPERATURE', 'TIDALVOLUME', 'VENTILATIONRATE', 'VENTILATOR',
       'HADM_ID', 'Unnamed: 0', 'SUBJECT_ID', '30readflag'],
      dtype='object')

In [28]:
neo_bloog_gas_all_time_dropna = neo_bloog_gas_all_time.dropna(axis= 1, how = 'all')
pco2_score = neo_bloog_gas_all_time.dropna(axis= 1, how = 'all')['PO2']
pco2_score.value_counts()

44.0     56
47.0     42
41.0     42
43.0     40
48.0     38
         ..
232.0     1
212.0     1
200.0     1
19.0      1
274.0     1
Name: PO2, Length: 263, dtype: int64

In [29]:
pco2_score2 = pco2_score
pco2_score[(pco2_score2.values<50)]  = 3
pco2_score[(pco2_score2>=50) & (pco2_score2<= 65)] = 1
pco2_score[(pco2_score2>65)] = 3
#pco2_score[(pco2_score2>=30) & (pco2_score2<=50)] = 3
#pco2_score[np.isnan(pco2_score2)] = 1

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in less
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [30]:
pco2_score = pco2_score.fillna(1)

In [31]:
neo_bloog_gas_all_time_dropna['o2_score'] = pco2_score

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
neo_basic_label_df_feature_SNAP = pd.read_csv('neo_los_basic_df.csv')
sub = pd.DataFrame(neo_bloog_gas_all_time_dropna[['o2_score','HADM_ID']])
neo_basic_label_df_feature_SNAP = util.merge(neo_basic_label_df_feature_SNAP,sub, 'inner','HADM_ID')
neo_basic_label_df_feature_SNAP.to_csv('o2_score.csv',index = False)
neo_basic_label_df_feature_SNAP

,Unnamed: 0,sum_los,HADM_ID,los_bin,o2_score
0,0,0.3638,100023,0,1.0
1,1,3.0070,100025,0,1.0
2,2,30.5118,100029,1,1.0
3,3,65.7146,100044,1,3.0
4,4,3.9160,100055,0,3.0
...,...,...,...,...,...
7969,7987,48.2892,199913,1,3.0
7970,7988,19.3016,199917,0,3.0
7971,7989,103.9422,199918,1,3.0
7972,7990,0.1066,199954,0,1.0


In [33]:
cm00 = np.sum((neo_basic_label_df_feature_SNAP['los_bin']==0)& (neo_basic_label_df_feature_SNAP['o2_score']<=1))
cm00
cm01 = np.sum((neo_basic_label_df_feature_SNAP['los_bin']==0)&(neo_basic_label_df_feature_SNAP['o2_score']>1))
cm00
cm10 = np.sum((neo_basic_label_df_feature_SNAP['los_bin']==1)&(neo_basic_label_df_feature_SNAP['o2_score']<=1))
cm10
cm11 = np.sum((neo_basic_label_df_feature_SNAP['los_bin']==1)&(neo_basic_label_df_feature_SNAP['o2_score']>1))
cm11

cm = np.array([[cm00,cm01],[cm10,cm11]])

In [34]:
util.chi2(cm)

6004.0
7974.0
[[4746.57837973 1257.42162027]
 [1557.42162027  412.57837973]]
chi 1685.7004726396872
pval 0.0


(1685.7004726396872, 0.0)

In [35]:
559/(7302+599)

0.07075053790659411

In [36]:
11/(216+11)

0.048458149779735685